In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [2]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
datasets = ["PXD001468","PXD000561","PXD000815","PXD001250","TMTCPTAC"]
#datasets = ["PXD001468"]


 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [3]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

In [4]:
results = {}
for datas in datasets:
    for suffix in ["","_nocorr_nort","_closed"]:
        dataset = datas + suffix
        print(dataset)
        #Processing dataset
        if "PXD001468" in dataset:
            def get_scan_ionbot(x):
                return int(x.split('.')[1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        else:
            def get_scan_ionbot(x):
                print(x)
                return int(x.split('=')[-1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        folder2 = folder + "/" + dataset
        ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
        ionbot["rank"] = ["first"]*len(ionbot)
        tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
        tmp["rank"] = ["lower"]*len(tmp)
        ionbot = pd.concat([ionbot,tmp])
        #NO IDEA WHAT HAPPENED HERE...
        ionbot = ionbot.drop_duplicates("ionbot_match_id")
        tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
        ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
        if "PXD001468" in dataset:
            ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
            ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
        else:
            ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
        ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] +  "XYX" +  ionbot["matched_peptide"]
        ionbot = ionbot[(ionbot["q-value"]<=0.01)]
        #ionbot["contains_missed_cleavage"] = ionbot["matched_peptide"].apply(contains_missed_cleavage)    
        #ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")
        #ionbot = ionbot[(ionbot["num_passed"]==1) |
        #        (ionbot["rank"]=="lower") |
        #        ((ionbot["num_passed"]>1) & (ionbot["rank"]=="first") & (ionbot["unexpected_modification"].isna()) & (ionbot["contains_missed_cleavage"]==False))]
        #need to re-assign best_psm
        #print(ionbot["num_passed"].value_counts())
        #best_psms = ionbot.sort_values("psm_score").drop_duplicates(["sfile","scan"], keep="last")
        #best_psm_ids = set(best_psms["ionbot_match_id"])
        #ionbot["rank"] = ionbot["ionbot_match_id"].apply(
        #lambda x: "first" if x in best_psm_ids else "lower")    
        results[dataset] = ionbot


PXD001468
PXD001468_nocorr_nort
PXD001468_closed
PXD000561
PXD000561_nocorr_nort
PXD000561_closed
PXD000815
PXD000815_nocorr_nort
PXD000815_closed
PXD001250
PXD001250_nocorr_nort
PXD001250_closed
TMTCPTAC
TMTCPTAC_nocorr_nort
TMTCPTAC_closed


In [9]:
def is_in_s(x):
    return x in s

to_write = []

for datas in datasets:
    dataset = datas
    print(dataset)
    #compute statistics

    #overlap between open and open without corr and rt information
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[(ionbot["rank"]=="first")&(ionbot["database"]=="T")]["PSM"])
    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]

    #first I count the overruled for target matches
    s2 = set(ionbot[(ionbot["rank"]=="first")&(ionbot["database"]=="T")]["PSM"])
    uniques2 = len(s2.difference(s1))

    #first I count the overruled for target matches
    s2_d = set(ionbot[(ionbot["rank"]=="first")&(ionbot["database"]=="D")]["PSM"])
    uniques2_d = len(s2_d.difference(s1))
    print("FDR lost with stats: %f"%(uniques2_d/uniques2))

    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))

    #second I count the overruled for decoy matches
    s2 = set(ionbot[(ionbot["rank"]=="first")&(ionbot["database"]=="D")]["PSM"])
    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled_decoy = len(tmp2.intersection(tmp3))
    print("FDR overruled by stats: %f" % (overruled_decoy/overruled))

    #overlap between open and closed
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[(ionbot["rank"]=="first")&(ionbot["database"]=="T")]["PSM"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    
    #first I count the overruled for target matches
    s2 = set(ionbot[ionbot["database"]=="T"]["PSM"])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)

    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))

    #second I count the overruled for decoy matches
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_closed"
    ionbot = results[dataset]

    s2 = set(ionbot[ionbot["database"]=="D"]["PSM"])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)

    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled_decoy = len(tmp2.intersection(tmp3))
    print("FDR overruled by open: %f"%(overruled_decoy/overruled))




PXD001468
FDR lost with stats: 0.211386
FDR overruled by stats: 0.120729
FDR overruled by open: 0.107622
PXD000561
FDR lost with stats: 0.235597
FDR overruled by stats: 0.124560
FDR overruled by open: 0.098659
PXD000815
FDR lost with stats: 0.243525
FDR overruled by stats: 0.132549
FDR overruled by open: 0.119109
PXD001250
FDR lost with stats: 0.059148
FDR overruled by stats: 0.052059
FDR overruled by open: 0.087723
TMTCPTAC
FDR lost with stats: 0.360096
FDR overruled by stats: 0.184171
FDR overruled by open: 0.166411


In [6]:
to_write = pd.DataFrame(to_write,columns=["unexpected","type","level","spectrum_file","search","value"])
to_write.to_csv("id_overlap.csv",index=False)

In [7]:
to_write

,unexpected,type,level,spectrum_file,search,value


In [8]:
for dataset in datasets:
    print(dataset)
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="PSM")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_closed")]["value"].sum()
    v2 = tmp[(tmp["type"]=="overlap_closed")&(tmp["search"]=="_closed")]["value"].sum()
    v = v2 / (v1+v2)
    print("lost in closed: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with RT: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="peptide")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))



PXD001468


ZeroDivisionError: division by zero